In [1]:
# !python train.py

In [2]:
import torch_geometric.datasets as datasets
from gnnexplain.nn.gcn import Model
import torch

cora = datasets.Planetoid(root='data', name='Cora', split='full', transform=None)
model = Model(cora)
checkpoint = torch.load('./checkpoints/cora-epoch=49-v1.ckpt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [3]:
from gnnexplain.model.gtree import Explainer
expl = Explainer()

In [18]:
from cProfile import Profile
import pstats

with Profile() as pr:
    expl.fit(
        cora,
        model,
        max_depth=10
        )
    pstats.Stats(pr) \
        .sort_stats(pstats.SortKey.CUMULATIVE) \
        .print_stats()

         159328 function calls (157322 primitive calls) in 5.418 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.005    0.003    5.416    5.416 /home/apluska/gnnexplain/gnnexplain/model/gtree.py:14(fit)
        8    0.200    0.025    5.215    0.652 /home/apluska/gnnexplain/gnnexplain/model/gtree.py:65(iterate)
        8    0.051    0.006    5.015    0.627 /home/apluska/gnnexplain/gnnexplain/model/gtree.py:82(_iterate)
       16    0.000    0.000    2.582    0.161 /home/apluska/miniconda3/envs/gnnexplain/lib/python3.12/site-packages/sklearn/base.py:1134(wrapper)
        8    0.001    0.000    2.569    0.321 /home/apluska/miniconda3/envs/gnnexplain/lib/python3.12/site-packages/sklearn/tree/_classes.py:221(_fit)
        8    2.541    0.318    2.541    0.318 {method 'build' of 'sklearn.tree._tree.DepthFirstTreeBuilder' objects}
      474    0.002    0.000    2.376    0.005 /home/apluska/miniconda3/envs/gnnexpla

In [13]:
expl.accuracy(cora, split='train'), expl.accuracy(cora, split='val'), expl.accuracy(cora, split='test')

(0.9561258278145696, 0.708, 0.732)

In [6]:
from torch_geometric.nn.aggr import Aggregation

isinstance(list(model.model.conv1.children())[1], Aggregation)

False